In [ ]:
# Core scverse libraries
import scanpy as sc
import anndata as ad
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# Integration
import scvi

# Opticlust clustering
import warnings
from opticlust.clust import clustering, clustering_plot
from opticlust.recommend import recommend_resolutions, score_resolutions
from opticlust.tree import clustree, clustree_plot

# Data retrieval
import pooch

# Set seed
import random
random.seed(1)

# Renaming clusters
from natsort import natsorted 

# Saving figures
import os

In [ ]:
# Figure settings and figure directory 
sc.settings.set_figure_params(dpi=250, facecolor="white", figsize = (12, 12))
sc.settings.figdir = 'YOUR_PATH_HERE'

In [ ]:
# Load in the Anndata object
adata = sc.read_h5ad('YOUR_PATH_HERE')
adata

In [ ]:
# Calculate QC metrics
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
# QC plots before filtering
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
    jitter=0.4,
    multi_panel=True,
)

sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
# Thresholds for filtering
pct_counts_mt_thres = 30 
n_genes_low_thres =  500 
n_genes_up_thres = 8000 
total_counts_low_thres = 1500 
total_counts_up_thres = 100000
sc.pp.scrublet(adata, batch_key='INPUT_BATCH_KEY_HERE') # identifies doublets

In [ ]:
# Filter cells based on thresholds
'Cells before filtering %d'%adata.n_obs

# Not used as there were no ribosome genes present in the data
#filter for percent ribo > 0.05
#adata = adata[adata.obs['pct_counts_ribo'] > 0.05, :]

# Filter for percent mito
adata = adata[adata.obs['pct_counts_mt'] < pct_counts_mt_thres, :]

# Filter for number of genes per cell
adata = adata[(adata.obs['n_genes_by_counts'] > n_genes_low_thres) & (adata.obs['n_genes_by_counts'] < n_genes_up_thres), :]

# Filter for number of features per cell
adata = adata[(adata.obs['total_counts'] > total_counts_low_thres) & (adata.obs['total_counts'] < total_counts_up_thres), :]

# Filter to exclude doublets
adata = adata[adata.obs['predicted_doublet'] == False, :]

'Remaining cells after filtering %d'%adata.n_obs

In [ ]:
# QC plots after filtering
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt", "pct_counts_ribo"],
    jitter=0.4,
    multi_panel=True,
)

sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
# Integration using scVI
# Identify highly variable genes
“sc.pp.highly_variable_genes(
    dataset, 
    flavor="seurat_v3", 
    n_top_genes=5000, 
    layer=None,  # "counts" in original
    batch_key="INPUT_BATCH_KEY_HERE",#,
    subset=True
)

# Initialize the scVI model
vae = scvi.model.SCVI(dataset, n_layers=2, n_latent=30, gene_likelihood="nb")

# Setting a manual seed for reproducibility (RTX 4080 super)
torch.manual_seed(42)

vae.train(max_epochs=150, early_stopping=True)

# Get the learned low-dimensional representation
dataset.obsm["X_scVI"] = vae.get_latent_representation()

# Compute neighborhood graph
sc.pp.neighbors(dataset, use_rep="X_scVI")

# Compute UMAP embedding based on neighborhood graph
sc.tl.umap(dataset)

In [ ]:
# Opticlust clustering
# Configuration
matplotlib.use("TkAgg")
plt.rcParams["font.family"] = "monospace"

# Clustering & plotting (updates adata.obs)
columns = clustering(adata, cluster_kwargs={"flavor":"leidenalg"})

# Score each resolution (updates adata.uns["opticlust"])
score_resolutions(adata, columns, tests="SH_CH_DB", method="mean",return_plot=True)
plt.savefig('/YOUR_PATH_HERE/recommendplot.png')

# To pre-select promising clustering resolutions, use:
tree_columns = clustering_plot(adata, columns, method="score", min_n_resolutions=1)
plt.savefig('/YOUR_PATH_HERE/clusteringplot.png')

# Divide the resolution into 3 bins and print the best resolution per bin
top_overall, top_low, top_medium, top_high = recommend_resolutions(adata, tree_columns)

# Build tree & plotting (updates adata.obs)
tree_data = clustree(adata, tree_columns, rename_cluster=True) #rename_cluster=False
clustree_plot(tree_data)
plt.savefig('/YOUR_PATH_HERE/clustreeplot.png')

# Plot the UMAPs for each resolution
sc.pl.umap(adata, color=tree_columns, legend_loc="on data", alpha=0.75, ncols=3)
plt.savefig('/YOUR_PATH_HERE/UMAPs.png')

In [ ]:
# Normalization
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
# Visualize UMAP
sc.pl.umap(
        adata,
        color= [''],
        wspace=0.5,
        size= 40,
        ncols = 1,
        title = '',
        legend_fontsize = '', 
        save = 'YOUR_PATH_HERE'
    )

In [ ]:
# Rename and plot distributions:
new_res_name= 'NEW_RES_NAME'

dict_cellstates = {'OLD_NUMBER': 'NEW_NAME/NUMBER', 'OLD_NUMBER': 'NEW_NAME/NUMBER', 'OLD_NUMBER': 'NEW_NAME/NUMBER'
    }

adata.obs[new_res_name] = adata.obs['OLD_RES_NAME'].cat.rename_categories(dict_cellstates)

adata.obs[new_res_name] = adata.obs[new_res_name].cat.set_categories(natsorted(list(set(dict_cellstates.values()))))

In [ ]:
# New UMAP colors
adata.uns["RES_NAME_colors"] = [
    '#COLOR_CODE', '#COLOR_CODE', '#COLOR_CODE', '#COLOR_CODE'
]

In [ ]:
# List of genes per cell type
cell_type = ['GENES']

# Plot them on UMAPs
plt.rcParams['axes.titlesize'] = 20
sc.pl.umap(
        adata,
        color= cell_type,
        wspace=0.5,
        size= 40,
        ncols = 4,
        save = 'NAME'
    )

In [ ]:
# Plot cell cycle
human_cell_cycle_genes = pd.read_csv('HUMAN_CELL_CYCLE_GENES.txt',header=None)
human_cell_cycle_genes
s_genes = human_cell_cycle_genes[0].to_list()[:43]
g2m_genes = human_cell_cycle_genes[0].to_list()[43:]

with open('human_g2m_genes.txt', "w") as fp:
    for item in g2m_genes:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

with open('human_s_genes.txt', "w") as fp:
    for item in s_genes:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

sc.pl.umap(
        adata,
        color= ['S_score', 'G2M_score'],
        wspace=0.5,
        size= 55,
        ncols = 1,
        legend_loc = 'on data',
    )

In [ ]:
fig = sc.pl.umap(
    adata, 
    color = ['RESOLUTION'], 
    groups = ['LIST_CLUSTERS_HIGHLIGHTED'], 
    wspace = 0.5,
    size = 40,
    ncols = 1,
    title = '',
    legend_fontsize = '',
    show = False,
    return_fig = True
)

# Modify legend labels using the first axis
ax = fig.axes[0]
legend_texts = ax.get_legend().get_texts()
for legend_text in legend_texts:
    if legend_text.get_text() == ['LIST_CLUSTERS_HIGHLIGHTED']:
        legend_text.set_text(['LIST_CLUSTERS_HIGHLIGHTED'])
    elif legend_text.get_text() == ['LIST_CLUSTERS_NOT_HIGHLIGHTED'] :
        legend_text.set_text(['LIST_CLUSTERS_NOT_HIGHLIGHTED'])

# Set title (optional, already handled by scanpy)
ax.set_title('', fontsize=20)

fig.savefig(os.path.join(sc.settings.figdir, "NAME.png"), dpi=250, bbox_inches='tight')

In [ ]:
# Generate HVG list for all relevant resolutions

# Define preselected resolutions
resolutions=['RESOLUTION']

# Check that adata has raw counts otherwise comment this out.
#adata.raw = adata

#del adata.raw

outputdir = 'YOUR_PATH_HERE'

# Extract relevant values from rank_genes_groups <cluster of interest> vs rest

joined_sig = pd.DataFrame()

for j in resolutions:
    new_res_name=j
    adata.uns['log1p']["base"] = None
    sc.tl.rank_genes_groups(adata, new_res_name, method='wilcoxon')

    resolution=new_res_name
    cond_list=adata.obs[resolution].to_list()
    cond_list=np.unique(cond_list).tolist()
    cond_list

    for i in cond_list:
        names = adata.uns["rank_genes_groups"]['names'][i]
        l2fc = adata.uns["rank_genes_groups"]['logfoldchanges'][i]
        padj = adata.uns["rank_genes_groups"]['pvals_adj'][i]
        scores = adata.uns["rank_genes_groups"]['scores'][i]

        A=['l2fc','padj','scores']
        B=names
        C=[l2fc,padj,scores]

        df =pd.DataFrame(C, columns=B)
        df.index=A
        df = df.T

        # Select only upregulated genes vs the others
        df_up = df[df["l2fc"]>1]
        df_up=df_up[df_up["padj"]<0.05]

        # Select down upregulated genes vs the others
        df_down = df[df["l2fc"]<-1]
        df_down=df_down[df_down["padj"]<0.05]

        #significant = df_up.append(df_down)
        significant = pd.concat([df_up, df_down], ignore_index=False)
        significant["cluster"] = i
        print(f"Highly variable genes vs others for condition: {i}")
        print(significant)
        joined_sig = pd.concat([joined_sig, significant], ignore_index=False)

        stripped_str = i.split(':')[0].strip()
    joined_sig.to_csv(f"{outputdir}HVG_{new_res_name}.txt", sep="\t", index=True)

In [ ]:
# Save the adata object
adata.write('/YOUR_PATH_HERE/NAME.h5ad')